# OSC Market Intelligence App - Component Testing

This notebook provides interactive testing of the main components of the Market Intelligence application, including comprehensive database testing.


## 1. Setup and Imports


In [5]:
# Import required modules
import sys
from pathlib import Path

# Ensure src is in the path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from src.config import DatabricksConfig, DatabaseConfig, AppConfig, OSC_COLORS, OSC_FONTS
from src.databricks_client import (
    get_workspace_client,
    call_endpoint,
    get_user_info,
    format_response,
)
from src.database import (
    init_database,
    create_conversation,
    add_message,
    update_message,
    get_conversation_messages,
    get_message_by_query_id,
    get_user_conversations,
)

print("✅ All imports successful!")


✅ All imports successful!


## 2. Test Configuration


In [6]:
# Load configurations
app_config = AppConfig.from_config()
databricks_config = DatabricksConfig.from_config()
database_config = DatabaseConfig.from_config()

print("App Configuration:")
print(f"  Title: {app_config.title}")
print(f"  Layout: {app_config.layout}")
print(f"  Async Queries: {app_config.async_queries_enabled}")
print()
print("Databricks Configuration:")
print(f"  Host: {databricks_config.host}")
print(f"  Endpoint: {databricks_config.endpoint_name}")
print()
print("Database Configuration:")
print(f"  Host: {database_config.host or 'Not configured'}")
print(f"  Database: {database_config.name}")
print(f"  Port: {database_config.port}")
print()
print("Note: Databricks authentication uses SDK defaults (~/.databrickscfg or environment)")
print("Note: Database credentials come from environment variables (DB_USER, DB_PASSWORD)")


App Configuration:
  Title: OSC Market Intelligence
  Layout: wide
  Async Queries: True

Databricks Configuration:
  Host: https://e2-demo-field-eng.cloud.databricks.com
  Endpoint: mas-1ab024e9-endpoint

Database Configuration:
  Host: instance-4401e5e2-c276-4302-8519-e9c3226ef66b.database.cloud.databricks.com
  Database: osc-delete-nov1
  Port: 5432

Note: Databricks authentication uses SDK defaults (~/.databrickscfg or environment)
Note: Database credentials come from environment variables (DB_USER, DB_PASSWORD)


## 3. Initialize & Test Database

This section will:
- Test database connectivity
- Create required tables if they don't exist
- Verify the schema


In [7]:
from databricks.sdk import WorkspaceClient

# Initialize workspace client using DatabricksConfig credentials
client = WorkspaceClient(
    host=databricks_config.host,
    # Optionally, you could pass token/databricks_cfg_path, but defaults should work if env/config present
)

print(f"✅ Databricks WorkspaceClient initialized for host: {databricks_config.host}")



✅ Databricks WorkspaceClient initialized for host: https://e2-demo-field-eng.cloud.databricks.com


In [8]:
import uuid
import psycopg2

user = client.current_user.me()
_user_email = user.emails[0].value
instance_name = database_config.name
instance = client.database.get_database_instance(name=instance_name)
_credential = client.database.generate_database_credential(
    request_id=str(uuid.uuid4()), instance_names=[instance_name]
)
_connection_params = {
    "host": instance.read_write_dns,
    "dbname": database_config.database,
    "user": _user_email,
    "password": _credential.token,
    "sslmode": "require",
}

conn = psycopg2.connect(**_connection_params)

AttributeError: 'DatabaseConfig' object has no attribute 'database'

In [ ]:
# Initialize and test database connection
print("=" * 60)
print("DATABASE INITIALIZATION & TESTING")
print("=" * 60)

# Check if database is configured
db_configured = bool(database_config.host and database_config.user and database_config.password)

if db_configured:
    print("\n✅ Database credentials are configured")
    print(f"   Host: {database_config.host}")
    print(f"   Port: {database_config.port}")
    print(f"   Database: {database_config.name}")
    print(f"   User: {database_config.user}")
    
    print("\n🔄 Testing database connection...")
    
    try:
        from src.database import get_connection
        
        # Test connection
        with get_connection(database_config):
            print("✅ Database connection successful!")
        
        print("\n🔧 Creating/verifying database schema...")
        print("   This will create tables if they don't exist:")
        print("   - conversations (stores conversation sessions)")
        print("   - messages (stores Q&A pairs with status tracking)")
        
        # Initialize database (creates tables if they don't exist)
        init_database(database_config)
        
        print("\n✅ Database schema initialized successfully!")
        print("\nDatabase tables created/verified:")
        print("   📋 conversations")
        print("      - id (primary key)")
        print("      - user_id")
        print("      - created_at")
        print("   📋 messages")
        print("      - id (primary key)")
        print("      - conversation_id (foreign key)")
        print("      - user_id")
        print("      - question")
        print("      - answer")
        print("      - status (pending/complete)")
        print("      - query_id (for async queries)")
        print("      - created_at")
        print("      - updated_at")
        print("   📋 indexes")
        print("      - idx_messages_conversation_id")
        print("      - idx_messages_query_id")
        
        db_configured = True
        
    except Exception as e:
        print(f"\n❌ Database error: {e}")
        print("\nPossible issues:")
        print("   - Database host not accessible")
        print("   - Incorrect credentials")
        print("   - Database does not exist")
        print("   - Insufficient permissions")
        print("\nApp will continue without database (no conversation history)")
        db_configured = False
        
else:
    print("\n⚠️  Database is not configured")
    print("\nTo enable conversation history, set these environment variables:")
    print("   export DB_USER=your_db_user")
    print("   export DB_PASSWORD=your_db_password")
    print("\nAnd configure in config.yaml:")
    print("   database:")
    print("     host: your_lakebase_host")
    print("     port: 5432")
    print("     name: market_intelligence")
    print("\nApp will continue without database (no conversation history)")

print("\n" + "=" * 60)


DATABASE INITIALIZATION & TESTING

⚠️  Database is not configured

To enable conversation history, set these environment variables:
   export DB_USER=your_db_user
   export DB_PASSWORD=your_db_password

And configure in config.yaml:
   database:
     host: your_lakebase_host
     port: 5432
     name: market_intelligence

App will continue without database (no conversation history)



## 4. Test Database CRUD Operations

Comprehensive testing of all database operations.


In [ ]:
# Test database operations - CRUD operations
if db_configured:
    print("=" * 60)
    print("TESTING DATABASE OPERATIONS")
    print("=" * 60)
    
    try:
        # 1. Create a test conversation
        print("\n1️⃣  Creating test conversation...")
        test_user_id = "test_user@example.com"
        conversation_id = create_conversation(database_config, test_user_id)
        print(f"   ✅ Created conversation with ID: {conversation_id}")
        
        # 2. Add a test message
        print("\n2️⃣  Adding test message...")
        message_id = add_message(
            database_config,
            conversation_id=conversation_id,
            user_id=test_user_id,
            question="What is the market outlook for 2024?",
            answer="The market shows positive indicators with steady growth expected...",
            status="complete"
        )
        print(f"   ✅ Added message with ID: {message_id}")
        
        # 3. Add a pending message (simulating async query)
        print("\n3️⃣  Adding pending message (async simulation)...")
        pending_message_id = add_message(
            database_config,
            conversation_id=conversation_id,
            user_id=test_user_id,
            question="Analyze trade execution patterns?",
            status="pending",
            query_id="test-query-123"
        )
        print(f"   ✅ Added pending message with ID: {pending_message_id}")
        
        # 4. Update the pending message
        print("\n4️⃣  Updating pending message...")
        update_message(
            database_config,
            message_id=pending_message_id,
            answer="Trade execution analysis complete: patterns show...",
            status="complete"
        )
        print(f"   ✅ Updated message {pending_message_id} to complete")
        
        # 5. Retrieve all messages in conversation
        print("\n5️⃣  Retrieving conversation messages...")
        messages = get_conversation_messages(database_config, conversation_id)
        print(f"   ✅ Retrieved {len(messages)} message(s)")
        
        print("\n   📋 Conversation History:")
        for i, msg in enumerate(messages, 1):
            print(f"\n   Message {i} (ID: {msg['id']})")
            print(f"   ├─ Question: {msg['question'][:60]}...")
            print(f"   ├─ Answer: {msg['answer'][:60] if msg['answer'] else 'N/A'}...")
            print(f"   ├─ Status: {msg['status']}")
            print(f"   ├─ Query ID: {msg['query_id'] or 'N/A'}")
            print(f"   └─ Created: {msg['created_at']}")
        
        # 6. Test query_id lookup
        print("\n6️⃣  Testing query_id lookup...")
        message_by_query = get_message_by_query_id(database_config, "test-query-123")
        if message_by_query:
            print(f"   ✅ Found message by query_id: {message_by_query['id']}")
        else:
            print("   ⚠️  No message found for query_id")
        
        # 7. Get user conversations
        print("\n7️⃣  Retrieving user conversations...")
        conversations = get_user_conversations(database_config, test_user_id)
        print(f"   ✅ User has {len(conversations)} conversation(s)")
        
        for conv in conversations:
            print(f"\n   Conversation {conv['id']}:")
            print(f"   ├─ Messages: {conv['message_count']}")
            print(f"   └─ Created: {conv['created_at']}")
        
        print("\n" + "=" * 60)
        print("✅ ALL DATABASE TESTS PASSED!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n❌ Error testing database operations: {e}")
        import traceback
        traceback.print_exc()
        print("\n" + "=" * 60)
else:
    print("\n⚠️ Skipping database tests - database not configured")
    print("Set DB_USER and DB_PASSWORD environment variables and configure database in config.yaml")



⚠️ Skipping database tests - database not configured
Set DB_USER and DB_PASSWORD environment variables and configure database in config.yaml


# OSC Market Intelligence App - Component Testing

This notebook provides interactive testing of the main components of the Market Intelligence application.


In [ ]:
## 1. Setup and Imports
import sys
from pathlib import Path

# Ensure src is in the path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from src.config import DatabricksConfig, DatabaseConfig, AppConfig, OSC_COLORS, OSC_FONTS
from src.databricks_client import (
    get_workspace_client,
    call_endpoint,
    get_user_info,
    format_response,
)
from src.database import (
    init_database,
    create_conversation,
    add_message,
    update_message,
    get_conversation_messages,
    get_user_conversations,
)

print("✅ All imports successful!")


✅ All imports successful!


In [ ]:
## 2. Test Configuration
app_config = AppConfig.from_config()
databricks_config = DatabricksConfig.from_config()
database_config = DatabaseConfig.from_config()

print("App Configuration:")
print(f"  Title: {app_config.title}")
print(f"  Layout: {app_config.layout}")
print(f"  Async Queries: {app_config.async_queries_enabled}")
print()
print("Databricks Configuration:")
print(f"  Host: {databricks_config.host}")
print(f"  Endpoint: {databricks_config.endpoint_name}")
print()
print("Database Configuration:")
print(f"  Host: {database_config.host or 'Not configured'}")
print(f"  Database: {database_config.name}")
print(f"  Port: {database_config.port}")
print()
print("Note: Databricks authentication uses SDK defaults (~/.databrickscfg or environment)")
print("Note: Database credentials come from environment variables (DB_USER, DB_PASSWORD)")


App Configuration:
  Title: OSC Market Intelligence
  Layout: wide
  Async Queries: True

Databricks Configuration:
  Host: https://e2-demo-field-eng.cloud.databricks.com
  Endpoint: mas-1ab024e9-endpoint

Database Configuration:
  Host: instance-4401e5e2-c276-4302-8519-e9c3226ef66b.database.cloud.databricks.com
  Database: databricks_postgres
  Port: 5432

Note: Databricks authentication uses SDK defaults (~/.databrickscfg or environment)
Note: Database credentials come from environment variables (DB_USER, DB_PASSWORD)


In [ ]:
## 3. Test Databricks Authentication
try:
    client = get_workspace_client()
    print("✅ Databricks client initialized successfully")
    
    # Get user info
    user_info = get_user_info(client)
    print(f"\nUser Information:")
    print(f"  User ID: {user_info['user_id']}")
    print(f"  Display Name: {user_info['display_name']}")
    print(f"  Active: {user_info['active']}")
except Exception as e:
    print(f"❌ Error initializing Databricks client: {e}")
    print("\nMake sure you have configured your Databricks credentials in .env file")


✅ Databricks client initialized successfully

User Information:
  User ID: scott.mckean@databricks.com
  Display Name: Scott McKean
  Active: True


In [ ]:
## 4. Test Endpoint Call
test_question = "Summarize the Trade execution patterns and suspicious activity rates in the past 24 hours"

try:
    print(f"Sending question: {test_question}")
    print("\nCalling endpoint...")
    
    # The call_endpoint function now handles the host configuration internally
    response = call_endpoint(
        databricks_config,
        test_question
    )
    
    print("\n✅ Response received!")
    print("\nRaw response:")
    print(response)
    
    # Format the response
    formatted_answer = format_response(response)
    print("\nFormatted answer:")
    print(formatted_answer)
    
except Exception as e:
    print(f"\n❌ Error calling endpoint: {e}")
    print("\nThis could be due to:")
    print("  - Endpoint not running")
    print("  - Incorrect endpoint name in config.yaml")
    print("  - Network connectivity issues")
    print("  - Authentication problems")


Sending question: Summarize the Trade execution patterns and suspicious activity rates in the past 24 hours

Calling endpoint...


KeyboardInterrupt: 

In [ ]:
## 5. Test Database Operations
# Check if database is configured
db_configured = bool(database_config.host and database_config.user)

if db_configured:
    print("✅ Database is configured")
    print("\nInitializing database schema...")
    
    try:
        init_database(database_config)
        print("✅ Database schema initialized successfully")
    except Exception as e:
        print(f"❌ Error initializing database: {e}")
        db_configured = False
else:
    print("⚠️ Database is not configured")
    print("Configure database credentials in .env to test database operations")


⚠️ Database is not configured
Configure database credentials in .env to test database operations


In [ ]:
# Test conversation creation and message storage
if db_configured:
    try:
        # Create a test conversation
        test_user_id = "test_user@example.com"
        conversation_id = create_conversation(database_config, test_user_id)
        print(f"✅ Created conversation with ID: {conversation_id}")
        
        # Add a test message
        message_id = add_message(
            database_config,
            conversation_id=conversation_id,
            user_id=test_user_id,
            question="What is the market outlook for 2024?",
            answer="The market shows positive indicators...",
            status="complete"
        )
        print(f"✅ Added message with ID: {message_id}")
        
        # Retrieve messages
        messages = get_conversation_messages(database_config, conversation_id)
        print(f"\n✅ Retrieved {len(messages)} message(s)")
        
        for msg in messages:
            print(f"\nMessage ID: {msg['id']}")
            print(f"  Question: {msg['question']}")
            print(f"  Answer: {msg['answer']}")
            print(f"  Status: {msg['status']}")
            print(f"  Created: {msg['created_at']}")
        
        # Get user conversations
        conversations = get_user_conversations(database_config, test_user_id)
        print(f"\n✅ User has {len(conversations)} conversation(s)")
        
    except Exception as e:
        print(f"❌ Error testing database operations: {e}")
else:
    print("⚠️ Skipping database tests - database not configured")


⚠️ Skipping database tests - database not configured


## 6. Test Response Formatting


In [ ]:
# Test different response formats
test_responses = [
    {
        "name": "Predictions format",
        "response": {"predictions": "This is a test prediction response"}
    },
    {
        "name": "OpenAI format",
        "response": {
            "choices": [
                {"message": {"content": "This is an OpenAI-style response"}}
            ]
        }
    },
    {
        "name": "Answer format",
        "response": {"answer": "This is a simple answer response"}
    },
    {
        "name": "Custom format",
        "response": {"custom_field": "Custom response", "metadata": {"score": 0.95}}
    }
]

print("Testing response formatting:")
print("=" * 60)

for test in test_responses:
    formatted = format_response(test["response"])
    print(f"\n{test['name']}:")
    print(f"  Input: {test['response']}")
    print(f"  Output: {formatted}")


Testing response formatting:

Predictions format:
  Input: {'predictions': 'This is a test prediction response'}
  Output: This is a test prediction response

OpenAI format:
  Input: {'choices': [{'message': {'content': 'This is an OpenAI-style response'}}]}
  Output: This is an OpenAI-style response

Answer format:
  Input: {'answer': 'This is a simple answer response'}
  Output: This is a simple answer response

Custom format:
  Input: {'custom_field': 'Custom response', 'metadata': {'score': 0.95}}
  Output: {
  "custom_field": "Custom response",
  "metadata": {
    "score": 0.95
  }
}


## 7. End-to-End Test

Test the complete flow: ask a question, get a response, and save to database.


In [ ]:
# Complete workflow test
if db_configured:
    try:
        print("Starting end-to-end test...\n")
        
        # 1. Create a new conversation
        user_id = user_info['user_id']
        conv_id = create_conversation(database_config, user_id)
        print(f"1. ✅ Created conversation: {conv_id}")
        
        # 2. Ask a question
        question = "What are the main regulatory priorities for 2024?"
        print(f"2. 📝 Question: {question}")
        
        # 3. Call the endpoint
        print("3. 🔄 Calling Databricks endpoint...")
        response = call_endpoint(client, databricks_config, question)
        answer = format_response(response)
        print(f"   ✅ Got response")
        
        # 4. Save to database
        msg_id = add_message(
            database_config,
            conversation_id=conv_id,
            user_id=user_id,
            question=question,
            answer=answer,
            status="complete"
        )
        print(f"4. ✅ Saved message: {msg_id}")
        
        # 5. Retrieve and display
        messages = get_conversation_messages(database_config, conv_id)
        print(f"5. ✅ Retrieved conversation history")
        
        print("\n" + "=" * 60)
        print("CONVERSATION SUMMARY")
        print("=" * 60)
        for msg in messages:
            print(f"\n🤔 Question: {msg['question']}")
            print(f"\n💡 Answer: {msg['answer']}")
            print(f"\n📊 Status: {msg['status']}")
            print(f"⏰ Time: {msg['created_at']}")
        
        print("\n" + "=" * 60)
        print("✅ END-TO-END TEST SUCCESSFUL!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n❌ End-to-end test failed: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ Skipping end-to-end test - requires database configuration")


⚠️ Skipping end-to-end test - requires database configuration


## 8. Summary and Next Steps


In [ ]:
print("Component Testing Summary")
print("=" * 60)
print("\n✅ Completed Tests:")
print("   - Configuration loading")
print("   - OSC branding setup")
print("   - Databricks authentication")
print("   - User information retrieval")
print("   - Endpoint calling")
print("   - Response formatting")
if db_configured:
    print("   - Database operations")
    print("   - Conversation management")
    print("   - Message storage and retrieval")
    print("   - End-to-end workflow")
else:
    print("   ⚠️  Database tests skipped (not configured)")

print("\n📚 Next Steps:")
print("   1. Run the Streamlit app: uv run streamlit run app.py")
print("   2. Test the full UI with real questions")
print("   3. Configure database for conversation history (if not done)")
print("   4. Deploy to Databricks Apps for production use")
print("\n" + "=" * 60)


Component Testing Summary

✅ Completed Tests:
   - Configuration loading
   - OSC branding setup
   - Databricks authentication
   - User information retrieval
   - Endpoint calling
   - Response formatting
   ⚠️  Database tests skipped (not configured)

📚 Next Steps:
   1. Run the Streamlit app: uv run streamlit run app.py
   2. Test the full UI with real questions
   3. Configure database for conversation history (if not done)
   4. Deploy to Databricks Apps for production use



---

## Additional Testing

Use the cells below for ad-hoc testing and experimentation.


In [ ]:
# Scratch cell for custom testing
